In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## IEX Constants
iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

## Redundant Assignment but improves Readability throughout code 
real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [3]:
token_status = test_token ## Set to either real token or test token for IEX

In [4]:
## Load IEX to get ETF statistics 
def get_IEX_statistics(stock_ticker, token=token_status):
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/stats/?token='+test_token)
        data_json = resp_data.json()
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/stats/?token='+real_token)
        data_json = resp_data.json()
        
    return data_json

In [5]:
## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [6]:
## Read in leftover data_dict 
path = Path('../Resources/leftover_data_dict.pkl')
leftover_data_dict = load_obj(path)

In [7]:
len(leftover_data_dict)

8375

In [8]:
del leftover_data_dict['nan']

In [9]:
## Make key_list 
leftover_key_list = []
for key in leftover_data_dict.keys():
    leftover_key_list.append(key)

In [10]:
leftover_key_list[0:10]

['AAA',
 'AAAU',
 'AAC',
 'AACG',
 'AACIU',
 'AACQ',
 'AACQU',
 'AACQW',
 'AADI',
 'AADR']

In [11]:
## Placeholder variables
time_series_df = 0
FMP_company_profile_json = 0
FMP_float_data_df = 0

## Data dict structure to refer to 
data = {
	'Fundamentals':                    			##
	{
		'sharesOutstanding': 17213236419,		## Sourced from IEX for ETFs, FMP for Equity
		 'floatShares': 16389334347.0,			## Can't source for ETFs at the moment - can get from FMP for Equity
		 'debt_ratio': 'debt_ratioValue',		## Blank - no source yet 
		 'exchange': 'Nasdaq Global Select',    ## Sourced from FMP 
		 'final_close_price': 149.800003,		## Sourced from iex_statistics
		 'peRatio': 14.407047826172997,			## Sourced from iex_statistics, =0 for ETFs
		 'beta': 1.4575864187912306,			## Sourced from iex_statistics for Equity, =0 for ETFs
		 'week52high': 169.1,					## Sourced from iex_statistics
		 'week52low': 118.18,					## Sourced from iex_statistics
		 'week52change': 0.4124429632647193,    ## Sourced from iex_statistics
		 'avg10Volume': 102534515,				## Sourced from iex_statistics
		 'avg30Volume': 79254408,				## Sourced from iex_statistics
		 'marketcap_IEX': 2663047538957,		## Sourced from iex_statistics
		 'marketcap_FMP': 2656852508672			## Sourced from FMP	
	},
	'dataFrame':time_series_df,							## FMP historical merged with Nasdaq Short Data and SEC FTD Data
	'companyProfile':FMP_company_profile_json,         	## Sourced from FMP, otherwise =0
	'floatData':FMP_float_data_df,   					## Sourced from FMP, otherwise =0
	'textNews':['article1','article2','article3'],    	## Not sourced
	'returns':'returns_data',    						## Not calculated
	'iex_statistics':'iex_statistics_data_json'     	## Added to completed data_dicts, both ETFs and Equity
}

In [12]:
# ## Test get_IEX_statistics
# test_symbol = leftover_key_list[0]
# symbol = test_symbol
# iex_etf_stats = get_IEX_statistics(symbol)
# iex_etf_stats

In [13]:
test_time_series_df = leftover_data_dict['AAC']
length_ = len(test_time_series_df)
test_time_series_df.close[length_-1]

9.84

In [14]:
## Create for loop that makes a new data_dict to add dataFrames to, 
## then add IEX data to all possible symbols 

iex_token = test_token
new_data_dict = {}
failed_list = [] 

for i in leftover_key_list:

    ## Load time_series data 
    time_series_df = leftover_data_dict[i]
    length_ = len(time_series_df)
    bool_var = time_series_df.isnull().values.any()
    if bool_var == True:
        failed_list.append(i)
        continue  ## Cancel loop and continue to next 
    
    iex_etf_stats = get_IEX_statistics(i,token=iex_token)
    sharesOutstanding = iex_etf_stats['sharesOutstanding']
    peRatio = iex_etf_stats['peRatio']
    beta = iex_etf_stats['beta']
    week52high = iex_etf_stats['week52high']
    week52low = iex_etf_stats['week52low']
    week52change = iex_etf_stats['week52change']
    avg10Volume = iex_etf_stats['avg10Volume']
    avg30Volume = iex_etf_stats['avg30Volume']
    marketcap_IEX = iex_etf_stats['marketcap']
    
    
    ## Initialize structure and add data
    data_structure = {
        'Fundamentals':                    			##
        {
            'sharesOutstanding': sharesOutstanding,		## Sourced from IEX for ETFs, FMP for Equity
             'floatShares': 0,			## Can't source for ETFs at the moment - can get from FMP for Equity
             'debt_ratio': 'debt_ratioValue',		## Blank - no source yet 
             'exchange': 0,             ## Sourced from FMP 
             'final_close_price': time_series_df.close[length_-1],		## Sourced from last value of dataFrame 
             'peRatio': peRatio,			    ## Sourced from iex_statistics, =0 for ETFs
             'beta': beta,			        ## Sourced from iex_statistics for Equity, =0 for ETFs
             'week52high': week52high,					## Sourced from iex_statistics
             'week52low': week52low,					## Sourced from iex_statistics
             'week52change': week52change,         ## Sourced from iex_statistics
             'avg10Volume': avg10Volume,				## Sourced from iex_statistics
             'avg30Volume': avg30Volume,				## Sourced from iex_statistics
             'marketcap_IEX': marketcap_IEX,		## Sourced from iex_statistics
             'marketcap_FMP': 0			## Sourced from FMP	
        },
        'dataFrame':time_series_df,	## FMP historical merged with Nasdaq Short Data and SEC FTD Data
        'companyProfile':0,         	## Sourced from FMP, otherwise =0
        'floatData':0,   					## Sourced from FMP, otherwise =0
        'textNews':['article1','article2','article3'],    	## Not sourced
        'returns':'returns_data',    						## Not calculated
        'iex_statistics':iex_etf_stats     	## Added to completed data_dicts, both ETFs and Equity
    }
    
    new_data_dict[i] = data_structure
    

KeyboardInterrupt: 

In [ ]:
len(new_data_dict)

In [ ]:
# ## Export IEX data from successful stuff
# path = Path('../Resources/test_05_2_iex_stat_export_.pkl')
# save_obj(new_data_dict,path)